In [11]:
# Section 1: Import Libraries
import requests  # This library helps us fetch data from APIs
import pandas as pd  # Pandas is used for handling and analyzing data
import numpy as np  # For numerical operations
from sklearn.model_selection import train_test_split  # This function splits data into training and testing sets
from sklearn.preprocessing import LabelEncoder  # Converts categorical data into numerical values
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor  # Models for classification and regression tasks
from sklearn.metrics import mean_squared_error  # This function measures the accuracy of our predictions
from datetime import datetime, timedelta  # These classes handle dates and times
import pytz

# Setup API Key and Base URL
BASE_URL = 'http://api.openweathermap.org/data/2.5/weather'
API_KEY = '10f9ed5e4b16f6a353e0b97c4c970c51'

# Function to fetch current weather data
def get_current_weather(city):
    url = f"{BASE_URL}?q={city}&appid={API_KEY}&units=metric"
    response = requests.get(url)
    data = response.json()

    return {
        'city': data['name'],
        'current_temp': round(data['main']['temp']),
        'feels_like': round(data['main']['feels_like']),
        'temp_min': round(data['main']['temp_min']),
        'temp_max': round(data['main']['temp_max']),
        'humidity': data['main']['humidity'],
        'description': data['weather'][0]['description'],
        'country': data['sys']['country'],
        'wind_gust_dir': data['wind']['deg'],
        'pressure': data['main']['pressure'],
        'Wind_Gust_Speed': data['wind']['speed']
    }


In [12]:
# Function to read and clean historical weather data
def read_historical_data(filename):
    df = pd.read_csv(filename)         # Read CSV file into DataFrame
    df.dropna(inplace=True)            # Remove rows with missing values
    df.drop_duplicates(inplace=True)   # Remove duplicate rows
    return df


In [18]:
def prepare_data(data):
    le = LabelEncoder()

    # Encode categorical columns with correct names
    data['WindGustDir'] = le.fit_transform(data['WindGustDir'])
    data['RainTomorrow'] = le.fit_transform(data['RainTomorrow'])

    # Define feature matrix X and target vector y with correct names
    X = data[['MinTemp', 'MaxTemp', 'WindGustDir', 'WindGustSpeed', 'Humidity', 'Pressure', 'Temp']]
    y = data['RainTomorrow']

    return X, y, le

In [13]:
# Function to train rain prediction model
def train_rain_model(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)

    predictions = model.predict(X_test)
    mse = mean_squared_error(y_test, predictions)
    print(f"Mean Squared Error for Rain Model: {mse:.2f}")

    return model


In [15]:
# Function to prepare regression data
def prepare_regression_data(data, feature):
    X, y = [], []

    for i in range(len(data) - 1):
        X.append(data[feature].iloc[i])
        y.append(data[feature].iloc[i + 1])

    return np.array(X).reshape(-1, 1), np.array(y)


In [7]:
# Function to train regression model
def train_regression_model(X, y):
    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X, y)
    return model


In [16]:
# Function to predict future values
def predict_future(model, current_value):
    predictions = [current_value]
    for i in range(5):  # Predict for the next 5 time points
        next_value = model.predict(np.array(predictions[-1]).reshape(1, -1))
        predictions.append(next_value[0])
    return predictions[1:]  # Exclude the initial value

In [22]:
def weather_view():
    city = input("Enter city name: ")
    current_weather = get_current_weather(city)  # assign the current_weather variable first
    historical_data = read_historical_data('weather.csv')

    # Prepare data for the rain prediction model
    X, y, le = prepare_data(historical_data)

    # Train the rain prediction model
    rain_model = train_rain_model(X, y)

    # Get the wind gust direction and normalize it
    wind_deg = current_weather['wind_gust_dir'] % 360

    compass_points = [
        ("N", 0, 11.25), ("NNE", 11.25, 33.75), ("NE", 33.75, 56.25),
        ("ENE", 56.25, 78.75), ("E", 78.75, 101.25), ("ESE", 101.25, 123.75),
        ("SE", 123.75, 146.25), ("SSE", 146.25, 168.75), ("S", 168.75, 191.25),
        ("SSW", 191.25, 213.75), ("SW", 213.75, 236.25), ("WSW", 236.25, 258.75),
        ("W", 258.75, 281.25), ("WNW", 281.25, 303.75), ("NW", 303.75, 326.25),
        ("NNW", 326.25, 348.75)
    ]

    # Corrected indentation for the compass direction search:
    compass_direction = next(
        (point for point, start, end in compass_points if start <= wind_deg < end),
        None
    )

    # Encoding the compass direction
    compass_direction_encoded = le.transform([compass_direction])[0] if compass_direction in le.classes_ else -1

    # Creating a data dictionary for the current weather with correct names
    current_data = {
        'MinTemp': current_weather['temp_min'],
        'MaxTemp': current_weather['temp_max'],
        'WindGustDir': compass_direction_encoded,
        'WindGustSpeed': current_weather['Wind_Gust_Speed'], # Assuming this key is correct in get_current_weather
        'Humidity': current_weather['humidity'],
        'Pressure': current_weather['pressure'],
        'Temp': current_weather['current_temp'],  # Assuming this key is correct in get_current_weather
    }

    current_df = pd.DataFrame([current_data])

    # Rain prediction
    rain_prediction = rain_model.predict(current_df)[0]

    # Prepare data for regression models
    X_temp, y_temp = prepare_regression_data(historical_data, 'Temp')  # Use 'temp' for temperature prediction
    X_hum, y_hum = prepare_regression_data(historical_data, 'Humidity')

    # Train regression models
    temp_model = train_regression_model(X_temp, y_temp)
    hum_model = train_regression_model(X_hum, y_hum)

    # Predict future temperature and humidity
    future_temp = predict_future(temp_model, current_weather['temp_min'])
    future_humidity = predict_future(hum_model, current_weather['humidity'])

    # Prepare for future time predictions
    timezone = pytz.timezone('Asia/Kolkata')
    now = datetime.now(timezone)
    next_hour = now + timedelta(hours=1)
    next_hour = next_hour.replace(minute=0, second=0, microsecond=0)

    future_times = [(next_hour + timedelta(hours=i)).strftime("%H:00") for i in range(5)]

    # Display the results
    print(f"City: {city}, {current_weather['country']}")
    # Change 'temp' to 'current_temp' to match the key in the dictionary
    print(f"Current Temperature: {current_weather['current_temp']}°C")
    print(f"Feels Like: {current_weather['feels_like']}°C")
    print(f"Minimum Temperature: {current_weather['temp_min']}°C")
    print(f"Maximum Temperature: {current_weather['temp_max']}°C")
    print(f"Humidity: {current_weather['humidity']}%")
    print(f"Weather Prediction: {current_weather['description']}")
    print(f"Rain Prediction: {'Yes' if rain_prediction else 'No'}")

    print("\nFuture Temperature Predictions:")
    for time, temp in zip(future_times, future_temp):
        print(f"{time}: {round(temp, 1)}°C")

    print("\nFuture Humidity Predictions:")
    for time, humidity in zip(future_times, future_humidity):
        print(f"{time}: {round(humidity, 1)}%")

# Call the function to execute the weather view
weather_view()

Enter city name: england
Mean Squared Error for Rain Model: 0.15
City: england, US
Current Temperature: 11°C
Feels Like: 11°C
Minimum Temperature: 11°C
Maximum Temperature: 11°C
Humidity: 76%
Weather Prediction: clear sky
Rain Prediction: No

Future Temperature Predictions:
11:00: 10.7°C
12:00: 11.9°C
13:00: 12.7°C
14:00: 13.7°C
15:00: 16.6°C

Future Humidity Predictions:
11:00: 61.6%
12:00: 56.5%
13:00: 40.7%
14:00: 52.9%
15:00: 51.0%
